In [40]:
import pandas as pd
import numpy as np

from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import GenericUnivariateSelect

from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import sparse

In [30]:
class MyBinaryRelevanceFeatureSelect():
   
    def fit(self, X, y):
        
        # I'm using a gaussian naive bayes base classifier
        self.BinaryRelevanceObject = BinaryRelevance(classifier = SVC(gamma= 'auto'), require_dense = [True, True])
        
        # fitting the data
        self.BinaryRelevanceObject.fit(X, y)
        
        #the classifiers for each label
        self.classifiers = self.BinaryRelevanceObject.classifiers_

        return self.classifiers
        
#     def partition(self):
#         return self.BinaryRelevanceObject.partition_#BinaryRelevanceObject
    
#     def model_count(self):
#         return self.BinaryRelevanceObject.model_count_

    def predict(self, X, y=None):
        return self.BinaryRelevanceObject.predict(X)
    
#     def feature_select(self, X, y):
        
#         #features_selected = []

# #         X_new = SelectKBest(chi2, k=2)

# #         # the feature selecting
# #         X_new.fit(X, y)

# #         # save indices of the saved attributes
# #         selected_attributes_indices = X_new.get_support(indices = True)
        
#         transformer = GenericUnivariateSelect(chi2, 'k_best', param=2)
#         X_new = transformer.fit_transform(X, y)
#         #X_new.shape
#         selected_attributes_indices = transformer.get_support(indices = True)
        
#         return selected_attributes_indices
    
    def feature_select(self, X, y, transformer):
        
        #transformer = SelectKBest(chi2, k=2)
        transformer.fit(X, y)
        selected_attributes_indices = transformer.get_support(indices = True)
        
        return selected_attributes_indices
    
    def sets_of_selected_features(self, X, predictions, classifier, transformer ): #X is the df with the predictions
        selected_features_array = []

        for i in predictions:
            indices_features_selected = clf.feature_select(X, predictions[i], transformer)
            selected_features_array.append(indices_features_selected)
    
        return selected_features_array

In [45]:
#making the dataset
X, y = make_multilabel_classification(n_classes=6, n_labels=2,sparse = True, allow_unlabeled=False, random_state=1)

#split in training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

#intializing wit the classifier class
clf = MyBinaryRelevanceFeatureSelect()

#fitting the data for training
clf.fit(X_train, y_train)

#sparse matrix
predictions = clf.predict(X_test)

#make a dataframe
x = pd.DataFrame(predictions.toarray())


#the feature selection
#x = clf.feature_select1(X_test,x[0],GenericUnivariateSelect(chi2, 'k_best', param=4))
#SelectKBest(chi2, k=5)
xx = clf.feature_select(X_test,x[0],SelectKBest(chi2, k=3))#[:]#array([ 1,  6, 14, 16, 19], dtype=int64)

In [46]:
arr = clf.sets_of_selected_features(X_test,x,clf,SelectKBest(chi2, k=3))
arr

[array([10, 15, 19], dtype=int64),
 array([17, 18, 19], dtype=int64),
 array([17, 18, 19], dtype=int64),
 array([ 3, 14, 19], dtype=int64),
 array([17, 18, 19], dtype=int64),
 array([17, 18, 19], dtype=int64)]

In [51]:
#type(x[0])

# np.intersect1d(x)
print(set(arr[0]).intersection(*arr))
print(set(arr[0]).union(*arr))
print(set(arr[0]).difference(*arr))
print(set(arr[0]).difference_update(*arr))


# for i in x:
#     print(i)

{19}
{3, 10, 14, 15, 17, 18, 19}
set()
None


In [16]:
feature_vector = []

for i in x:
    xx = clf.feature_select(X_test,x[i],SelectKBest(chi2, k=2))
    feature_vector.append(xx)
    
feature_vector
# while i < len(x):
#     print(x[i])
#     i = i + 1

# i = 0
# for i in range(0,len(x)):
#     print(x[i])

# for i in xx:
#     print

# for i in x:
#     print(i)

[array([ 5, 16], dtype=int64),
 array([18, 19], dtype=int64),
 array([18, 19], dtype=int64),
 array([ 7, 19], dtype=int64),
 array([11, 19], dtype=int64),
 array([18, 19], dtype=int64)]

In [17]:
feature_vector = []

for i in x:
    xx = clf.feature_select(X_test,x[i],GenericUnivariateSelect(chi2, 'k_best', param=2))
    feature_vector.append(xx)
    
feature_vector

[array([ 5, 16], dtype=int64),
 array([18, 19], dtype=int64),
 array([18, 19], dtype=int64),
 array([ 7, 19], dtype=int64),
 array([11, 19], dtype=int64),
 array([18, 19], dtype=int64)]

In [122]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

        # initialize Binary Relevance multi-label classifier
        # with an SVM classifier
        # SVM in scikit only supports the X matrix in sparse representation

classifier = BinaryRelevance(
            classifier = SVC(),
            require_dense = [False, True]
        )

        # train
classifier.fit(X_train, y_train)

        # predict
predictions = classifier.predict(X_test)

C:\Users\Abel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [215]:
pd.DataFrame(predictions.toarray()).head()

,0,1,2,3
0,1,1,0,1
1,1,1,0,1
2,1,1,0,1
3,1,1,0,1
4,1,1,0,0


In [216]:
pd.DataFrame(y_train).head()

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,1,1,1,1
3,0,1,0,0
4,0,1,0,0


In [20]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
print(y.shape)


lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(150, 4)
(150,)
(150, 3)


C:\Users\Abel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [40]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
X, y = load_digits(return_X_y=True)
X.shape

transformer = SelectKBest(chi2, k=5)
transformer.fit(X, y)
#X_new.shape
transformer.get_support(indices = True)

array([33, 34, 42, 43, 54], dtype=int64)